In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"

In [6]:
from imutils import face_utils
import numpy as np
import imutils, dlib, cv2, csv, glob
import matplotlib.pyplot as plt
import pandas as pd

In [10]:
#  동물 이미지에 랜드마크를 찍어 csv 파일로 저장

def show_raw_detection_plt(path, detector, predictor):
    img_path = glob.glob(path)
    
    for j in range(0, len(img_path)):
        img = cv2.imread(img_path[j])
    
        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        dets = detector(img, upsample_num_times = 1)
        img_result = img.copy()

        for d in dets:
            x1, y1 = d.rect.left(), d.rect.top()
            x2, y2 = d.rect.right(), d.rect.bottom()

            cv2.rectangle(img_result, (x1, y1), (x2, y2), (255, 0, 0), 2, cv2.LINE_AA)

        shapes = []
        point = []

        for i, d in enumerate(dets):
            shape = predictor(img, d.rect)
            shape = face_utils.shape_to_np(shape)

            for i, p in enumerate(shape):
                shapes.append(shape)

                cv2.circle(img_result, p, 3, (0, 0, 255), -1, cv2.LINE_AA)
                cv2.putText(img_result, str(i), tuple(p), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                           (255, 255, 255), 1, cv2.LINE_AA)

                p = p.astype('int')
                point.extend(p)

            with open('./dog/{}.csv'.format(img_path[j]), 'w', newline = '') as file:
                write = csv.writer(file)

                write.writerow(point)
            

        
#     plt.figure(figsize = (16, 16))
#     plt.imshow(img_result)

In [5]:
# 랜드마크를 전처리

def landmark_fix(landmarks):

    re_land = landmarks.copy()

    for i, j, k in zip(range(1, 7), [60, 60, 59, 59, 58, 58], range(18, 1, -3)):
        re_land[i] = (landmarks[j] - landmarks[i]) / k + landmarks[i]

    for i, j, k in zip([7, 8, 9], [57, 57, 57], [2, 2, 2]):
        re_land[i] = (landmarks[j] - landmarks[i]) / k + landmarks[i]

    for i, j, k in zip(range(15, 9, -1), [54, 54, 55, 55, 56, 56], range(18, 1, -3)):
        re_land[i] = (landmarks[j] - landmarks[i]) / k + landmarks[i]

In [ ]:
img_path = glob.glob('./test/*.*')

detector = dlib.cnn_face_detection_model_v1('dogHeadDetector.dat')
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

show_raw_detection_plt(img_path, detector, predictor)